In [ ]:
import matplotlib.pyplot as plt
import os
import numpy as np 
import pandas as pd

def load_sigma(subj, visual_area, task, atlas="benson"):
    base_path = f"{MAIN_PATH}/{subj}/ses-02/{atlas}/{task}/nordic"
    sigma_vals = []

    for hemi in hemis:
        path = f"{base_path}/GM/{hemi}/{visual_area}-V1/best_fits_prf.csv"
        df = pd.read_csv(path)
        df = df.dropna(subset=["Best Sigma Finer", "Best Variance Explained Finer", "Source Eccentricity"])
        df = df[(df["Best Sigma Finer"] > 0.05) & (df["Best Variance Explained Finer"] > 0.3)]
        sigma_vals.extend(df["Best Sigma Finer"].values)
    return sigma_vals

def plot_group_comparison_hist(visual_area, task, save_dir, atlas):
    """Plot group comparison using cumulative histograms (CDFs)."""
    fig, ax = plt.subplots(figsize=(6,6), layout="constrained")

    for group_name, subjects in groups.items():
        all_sigmas = []
        for subj in subjects:
            sigmas = load_sigma(subj, visual_area, task, atlas)
            #if sigmas:
            #    all_sigmas.extend(sigmas)

        if all_sigmas:
            color = 'red' if group_name == "POAG" else 'blue'
            ax.hist(all_sigmas, bins=1000, density=True, histtype="step", cumulative=True, 
                    label=f"{group_name} (N={len(subjects)})", linewidth=2,color=color)

    ax.set_title(f"{visual_area} - {task} (CDF)")
    ax.set_xlabel("Connective Field Sigma (mm)")
    ax.set_ylabel("Cumulative Probability")
    ax.set_xlim(0, 2)
    ax.set_ylim(0, 1)
    ax.grid(True)
    ax.legend()
    
    fig.savefig(os.path.join(save_dir, f"{visual_area}_{task}_group_comparison_CDF.jpg"), dpi=300)
    plt.close(fig)

def plot_group_comparison(visual_area, task, save_dir, atlas):
    """Plot group comparison median + CI."""
    plt.figure(figsize=(6,6))
    for group_name, subjects in groups.items():
        all_cdfs = []
        for subj in subjects:
            sigmas = load_sigma(subj, visual_area, task, atlas)
            sorted_sigma = np.sort(sigmas) # np.sort sort sigma values 
            y = np.cumsum(sorted_sigma).astype("float32")  # Cumulative sum of sorted values
            y/=y.max() # Normalize to 1 (max value becomes 1)
            y*=100. # Scale to percentage (0–100)
            y = np.hstack((0,y)) # Add a 0 at the start
            x = np.linspace(0, 100, y.size)  # Create corresponding x-values from 0 to 100
            cdf = np.linspace(0, 1, len(sorted_sigma)) # np.linspace creates a linear CDF from 0 to 1 for the sorted valu
            grid_cdf = np.interp(x, sorted_sigma, cdf, left=0, right=1) # np.interp basically connects the dots with straight lines over a common grid x so that all subjects’ can be averaged
            all_cdfs.append(grid_cdf)

        if all_cdfs:
            all_cdfs = np.array(all_cdfs)
            median = np.median(all_cdfs, axis=0) # Compute median and 25–75% confidence intervals
            p25 = np.percentile(all_cdfs, 25, axis=0)
            p75 = np.percentile(all_cdfs, 75, axis=0)
            color = 'red' if group_name == "POAG" else 'blue'
            plt.plot(x, median, color=color, linewidth=2.5, label=f"{group_name} (N={len(all_cdfs)})")
            plt.fill_between(x, p25, p75, color=color, alpha=0.2)

    plt.title(f"{visual_area} - {task}")
    plt.xlabel("Connective Field Sigma (mm)")
    plt.ylabel("Cumulative Probability")
    plt.grid(True)
    plt.xlim(0, 2)
    plt.legend()
    plt.savefig(os.path.join(save_dir, f"{visual_area}_{task}_group_comparison.jpg"))
    plt.close()

visual_area = 'V2'
task = 'RET'
atlas = 'manual'
hemis = ['lh', 'rh']
denoising = 'nordic'
poag = ["sub-18"]
hc = ["sub-46"]

groups = {"HC": hc, "POAG": poag}
save_dir = '/Volumes/FedericaCardillo/pre-processing/projects/PROJECT_EGRET-AAA/derivatives/CFM_previous'
MAIN_PATH = '/Volumes/FedericaCardillo/pre-processing/projects/PROJECT_EGRET-AAA/derivatives/CFM_previous'
plot_group_comparison_hist(visual_area, task, save_dir, atlas)

No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
